# Classic ESN

In [ ]:
import os
from os import listdir
from os.path import join
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import random
from functions import train, forecast
from research.plots.systems import plot_forecast
from research.plots.systems import plot_system

from src.model import *

# FIX: Standardize data
# BUG: transient 0
# TODO: use fit with 1 epoch and then reset model states
# NOTE: Separate system generation to other repo
# TODO: save dt in params.json


folder = '/home/dionisio35/Documents/thesis/data/MG/16.8/'
data_file = join(folder, random.choice(listdir(folder)))
dt=0.25
steps = 5

transient = 1000
train_length = 20000
forecast_length = 1000

input_scaling = 0.5 # input sigma
reservoir_degree = 8 # numero de conexiones de nodos del reservoir
spectral_radius = 0.9 # radio espectral de la matriz de pesos del reservoir
reservoir_sigma = 0.6 # sigma de la distribucion de pesos del reservoir
rewiring = 1 # probabilidad de reconexion en reservoir Watts-Strogatz

# FIX
units = 1000 # Numero de nodos del reservoir
seed = 21277854
reservoir_activation = 'tanh'

regularization = 1e-08
leak_rate = 0.1 # Cuanta informacion se filtra hacia el siguiente estado 0-1(toda-nada)

input_initializer = 'InputMatrix'
input_bias_initializer = 'RandomUniform'
reservoir_initializer = 'WattsStrogatzNX'

model = "ESN"

reservoir_amount=8 # Numero de reservoirs
overlap=2 # Numero de nodos que se comparten entre reservoirs

lyapunov_exponent=0.06693636363636364 # Exponente de Lyapunov de KS


In [ ]:
model = train(
    data_file=data_file,
    model=model,

    units=units,
    steps=steps,
    transient=transient,
    train_length=train_length,

    input_initializer=input_initializer,
    input_bias_initializer=input_bias_initializer,
    reservoir_initializer=reservoir_initializer,
    reservoir_activation=reservoir_activation,

    input_scaling=input_scaling,
    leak_rate=leak_rate,
    spectral_radius=spectral_radius,
    rewiring=rewiring,
    reservoir_degree=reservoir_degree,
    reservoir_sigma=reservoir_sigma,
    regularization=regularization,

    reservoir_amount=reservoir_amount,
    overlap=overlap,
    
    seed=seed,
)

# Get Reservoir Graph

In [ ]:
import keras.utils

a = keras.utils.model_to_dot(
    model.model,
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    subgraph=False,
    show_layer_activations=False,
    show_trainable=False,
)


In [ ]:
for i in listdir(folder):
    file = join(folder, i)
    _forecast, val_target = forecast(
        trained_model=model,
        transient=transient,
        train_length=train_length,
        data_file=file,
        forecast_method='classic',
        forecast_length=forecast_length,
        steps=steps,
    )
    plot_forecast(_forecast, val_target, show=True, cmap="jet", dt=dt)

# Add Metrics

In [ ]:
from research.grid.tools import calculate_metrics

calculate_metrics(
    results_path="/home/dionisio35/Documents/thesis/outputs/problem_with_pred/",
    data_path='/home/dionisio35/Documents/thesis/data/MG/16.8/'
)

# ECA

In [ ]:
from os import listdir
from os.path import join
import random

from functions import train
from cli.enums import *
from functions import forecast
from research.plots.systems import plot_forecast

folder = '/home/dionisio35/Documents/thesis/data/MG/16.8/'
data_file = join(folder, random.choice(listdir(folder)))

output_dir = "/home/dionisio35/Documents/thesis/model"
units =100
input_initializer = "InputMatrix"
input_bias_initializer = "RandomUniform"
input_scaling = 0.5
leak_rate = 0.1
reservoir_activation = "tanh"
seed = None
readout_layer = "linear"
regularization = 1e-8
transient = 1000
train_length = 10000
steps = 1
eca_rule = [0,1,1,1,0.0174,1,0,0]
eca_steps = 1
forecast_length = 1000
dt = 0.5



In [ ]:
model = train(
    data_file=data_file,
    output_dir=output_dir,
    model=EnumModel.ECA.value,
    units=units,
    input_initializer=input_initializer,
    input_bias_initializer=input_bias_initializer,
    input_scaling=input_scaling,
    leak_rate=leak_rate,
    reservoir_activation=reservoir_activation,
    seed=seed,
    readout_layer=readout_layer,
    regularization=regularization,
    transient=transient,
    train_length=train_length,
    steps=steps,
    eca_rule=eca_rule,
    eca_steps=eca_steps,
)

In [ ]:


for i in listdir(folder):
    file = join(folder, i)
    _forecast, val_target = forecast(
        trained_model=model,
        transient=transient,
        train_length=train_length,
        data_file=file,
        forecast_method='classic',
        forecast_length=10,
        steps=steps,
    )
    plot_forecast(_forecast, val_target, show=True, cmap="jet", dt=dt)